In [1]:
import argparse
import torch
import numpy as np
import pandas as pd
import gc
from tqdm.notebook import tqdm
from recbole.quick_start import load_data_and_model

In [2]:
# python run_inference.py --model_path=/opt/ml/input/RecBole/saved/SASRecF-Apr-07-2022_03-17-16.pth 로 실행
# python /home/RecBole/run_inference.py -m=/home/RecBole/saved/LightGCN-Dec-28-2022_06-04-00.pt
    
# args, _ = parser.parse_known_args()
# model, dataset 불러오기
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file = "/home/RecBole/saved/LightGCN-Feb-07-2023_19-15-05.pth"
)

del train_data, valid_data
gc.collect()

07 Feb 19:33    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2023
state = INFO
reproducibility = True
data_path = /home/RecBole/recbole/config/../dataset_example/ml-100k
checkpoint_dir = /home/RecBole/saved/
show_progress = True
save_dataset = True
dataset_save_path = saved_dataset
save_dataloaders = True
dataloaders_save_path = saved_dataloaders
log_wandb = True

Training Hyper Parameters:
epochs = 1000
train_batch_size = 4048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 1000
clip_grad_norm = None
weight_decay = 0.0005
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.675, 0.075, 0.25]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'MAP', 'Precision', 'GAUC', 'ItemCoverage', 'AveragePopularity', 'GiniIndex', 'ShannonEntrop

54

In [4]:
external_user_ids = dataset.id2token(dataset.uid_field, list(range(dataset.user_num)))[1:]
external_user_ids

array(['196', '186', '22', '244', '166', '298', '115', '253', '305', '6',
       '62', '286', '200', '210', '224', '303', '122', '194', '291',
       '234', '119', '167', '299', '308', '95', '38', '102', '63', '160',
       '50', '301', '225', '290', '97', '157', '181', '278', '276', '7',
       '10', '284', '201', '287', '246', '242', '249', '99', '178', '251',
       '81', '260', '25', '59', '72', '87', '42', '292', '20', '13',
       '138', '60', '57', '223', '189', '243', '92', '241', '254', '293',
       '127', '222', '267', '11', '8', '162', '279', '145', '28', '135',
       '32', '90', '216', '250', '271', '265', '198', '168', '110', '58',
       '237', '94', '128', '44', '264', '41', '82', '262', '174', '43',
       '84', '269', '259', '85', '213', '121', '49', '155', '68', '172',
       '19', '268', '5', '80', '66', '18', '26', '130', '256', '1', '56',
       '15', '207', '232', '52', '161', '148', '125', '83', '272', '151',
       '54', '16', '91', '294', '229', '36', '70', '

In [5]:
user_id2token = dataset.field2id_token['user_id']
item_id2token = dataset.field2id_token['item_id']

In [6]:
device = config.final_config_dict['device']

In [7]:
# user-item sparse matrix
matrix = dataset.inter_matrix(form='csr')

# user id, predict item id 저장 변수
pred_list = None
user_list = None

In [8]:
matrix

<944x1683 sparse matrix of type '<class 'numpy.float64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [24]:
model.eval()

for data in test_data:
    interaction = data[0].to(device)
    score = model.full_sort_predict(interaction)

    rating_pred = score.cpu().data.numpy().copy()
    
    print(rating_pred)
    print(rating_pred.shape)
    
    batch_user_index = interaction['user_id'].cpu().numpy()
    
    print(batch_user_index.shape)
    
    rating_pred[matrix[batch_user_index].toarray() > 0] = 0
    
    ind = np.argpartition(rating_pred, -10)[:, -10:]

    arr_ind = rating_pred[np.arange(len(rating_pred))[:, None], ind]

    arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]

    batch_pred_list = ind[
        np.arange(len(rating_pred))[:, None], arr_ind_argsort
    ]

    # 예측값 저장
    if pred_list is None:
        pred_list = batch_pred_list
        user_list = batch_user_index
    else:
        pred_list = np.append(pred_list, batch_pred_list, axis=0)
        user_list = np.append(user_list, batch_user_index, axis=0)

[-5.5160969e-09  1.4331990e+00  2.4582176e+00 ... -5.1355348e-03
 -3.6828227e-03 -2.1332261e-01]
(3366,)
(2,)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [10]:
result = []
for user, pred in zip(user_list, pred_list):
    for item in pred:
        result.append((int(user_id2token[user]), int(item_id2token[item])))

TypeError: zip argument #1 must support iteration

In [11]:
# 데이터 저장
dataframe = pd.DataFrame(result, columns=["user", "item"])
dataframe.sort_values(by='user', inplace=True)
dataframe.to_csv(
    "/saved/submission.csv", index=False
)

FileNotFoundError: [Errno 2] No such file or directory: '/saved/submission.csv'

In [27]:
def predict_item(external_user_id, dataset, model):
    """

    article_id_list => interaction_sequence recorded by Target User

    """
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset.inter_feat[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[index]
      
        test = {
                'article_id_list': add_last_item(
                    input_interaction, 
                    input_interaction['article_id'][-1].item(),
                    model.max_seq_length
                ),
                'item_length': torch.tensor(
                    [input_interaction['item_length'][-1].item() + 1
                                             if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        
        new_inter = Interaction(test) # 마지막 정보도 함께 추론에 사용된다.
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter) # 개별 모델 클래스 내부 메소드
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, 12)

In [28]:
""" 

This is For-Loop for Inference Target User's buying next Period

[Inference Step]
1) Range: [0, 25000] => Colab
2) Range: [25000, 50000] => Kaggle Notebook Stream
3) Range: [50000, 105333] => Kaggle Notebook Background

"""
outer_topk_item_list = []

for uuser in tqdm(external_user_ids):
    result = predict_item(uuser, test_data, model)
    inter_topk_items = result[-1][-1]
    outer_topk_item_list.append(dataset.id2token(dataset.iid_field, inter_topk_items.cpu()).tolist())

    del result, inter_topk_items
    gc.collect()

  0%|          | 0/943 [00:00<?, ?it/s]

AttributeError: 'FullSortEvalDataLoader' object has no attribute 'token2id'

In [ ]:
group_one_submission = pd.DataFrame(external_user_ids[50000:105333],
                                    columns=['customer_id'])
group_one_submission['prediction'] = [' '.join(outer_topk_item) for outer_topk_item in outer_topk_item_list]
group_one_submission.to_csv('(Range_3)final_group_submission.csv', index=False)
group_one_submission.head()